In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.environ['PINECONE_API_KEY']

# index 생성

In [3]:
from pinecone import Pinecone
# 클라이언트 생성
pinecone_client = Pinecone(api_key=PINECONE_API_KEY)

In [4]:
pinecone_client

In [5]:
from pinecone import ServerlessSpec
pinecone_client.create_index(
    name='embedding-3d',
    dimension=3,
    metric='cosine',
    spec=ServerlessSpec(
        cloud='aws',
        region='us-east-1'
    )
)

{
    "name": "embedding-3d",
    "metric": "cosine",
    "host": "embedding-3d-o45zd0f.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 3,
    "deletion_protection": "disabled",
    "tags": null
}

In [6]:
index = pinecone_client.Index('embedding-3d')
index

c:\Users\Admin\miniconda3\envs\prompting_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [14]:
index.describe_index_stats()

{'dimension': 3,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'embedding-3-ns1': {'vector_count': 6}},
 'total_vector_count': 6,
 'vector_type': 'dense'}

In [17]:
for ids in index.list(namespace='embedding-3d-ns1'):
    print(ids)

['vec1', 'vec2', 'vec3', 'vec4', 'vec5', 'vec6']


# pinecone data 형식

In [16]:
index.upsert(
    vectors= [
        {'id' :'vec1', 'values' :[1.0,1.5,2.0], 'metadata': {'genre':'drama'}},
        {'id' :'vec2', 'values' :[2.0,1.5,0.5], 'metadata': {'genre':'action'}},
        {'id' :'vec3', 'values' :[0.1,0.3,0.5], 'metadata': {'genre':'drama'}},
        {'id' :'vec4', 'values' :[5.0,10.0,5.1], 'metadata': {'genre':'horror'}},
        {'id' :'vec5', 'values' :[3.5,2.2,1.1], 'metadata': {'genre':'comedy'}},
        {'id' :'vec6', 'values' :[0.5,1.0,1.5], 'metadata': {'genre':'drama'}}
    ],
    namespace='embedding-3d-ns1'
) #update + insert

{'upserted_count': 6}

# query

In [24]:
response = index.query(
    namespace='embedding-3d-ns1',
    vector=[5.0,3.0,1.0],
    top_k=3,
    include_values=True,
    include_metadata=True,
    filter={'genre':{'$eq':'action'}}
)
response

{'matches': [{'id': 'vec2',
              'metadata': {'genre': 'action'},
              'score': 0.995270252,
              'values': [2.0, 1.5, 0.5]}],
 'namespace': 'embedding-3d-ns1',
 'usage': {'read_units': 1}}